In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import os

In [ ]:
def load_and_split_multimodal_data(
    sensor_csv_path: str,
    image_data_dir: str,
    test_size: float = 0.1,
    val_size: float = 0.2,
    batch_size: int = 32,
    img_size: tuple = (120, 160),
    random_state: int = 42
):
    """
    Load and split both sensor CSV and image folders into multimodal tf.data.Datasets.

    Args:
        sensor_csv_path: Path to the gas sensor CSV.
        image_data_dir: Base directory of image class subfolders.
        test_size: Fraction of data reserved for testing.
        val_size: Fraction of remaining data reserved for validation.
        batch_size: Batch size for tf.data pipelines.
        img_size: Size to resize images to (height, width).
        random_state: Seed for reproducibility.

    Returns:
        train_ds, val_ds, test_ds: tf.data.Dataset yielding ((sensor, image), label).
    """
    # 1) Load sensor CSV
    sdf = pd.read_csv(sensor_csv_path)
    sdf = sdf.drop(columns=["Serial Number"], errors='ignore')
    sdf['Gas'] = sdf['Gas'].astype('category').cat.codes

    # Build image filename column
    sdf['Image_File'] = sdf['Corresponding Image Name'].astype(str) + ".png"

    # Extract sensor features and labels
    sensor_cols = [c for c in sdf.columns if c not in ['Gas', 'Corresponding Image Name', 'Image_File']]
    sensors = sdf[sensor_cols].values.astype('float32')
    labels = sdf['Gas'].values.astype('int32')

    # Normalize sensors
    scaler = StandardScaler().fit(sensors)
    sensors = scaler.transform(sensors)

    # Map image names to full paths
    base = image_data_dir
    def find_path(fname):
        for cls in os.listdir(base):
            p = os.path.join(base, cls, fname)
            if os.path.exists(p):
                return p
        return None

    paths = np.array(sdf['Image_File'].map(find_path))
    valid = ~pd.isna(paths)

    sensors = sensors[valid]
    paths   = paths[valid]
    labels  = labels[valid]

    # Split multimodal arrays
    X_temp, X_test, S_temp, S_test, y_temp, y_test = train_test_split(
        paths, sensors, labels,
        test_size=test_size, stratify=labels, random_state=random_state
    )
    val_frac = val_size / (1 - test_size)
    X_train, X_val, S_train, S_val, y_train, y_val = train_test_split(
        X_temp, S_temp, y_temp,
        test_size=val_frac, stratify=y_temp, random_state=random_state
    )

    # Define loader
    def loader(path, sens, lab):
        img = tf.io.read_file(path)
        img = tf.image.decode_png(img, channels=3)
        img = tf.image.resize(img, img_size)
        img = tf.cast(img, tf.float32) / 255.0
        sens = tf.cast(sens, tf.float32)
        return (sens, img), lab

    # Build tf.data datasets
    def make_ds(paths, sensors, labels, shuffle=False):
        ds = tf.data.Dataset.from_tensor_slices((paths, sensors, labels))
        ds = ds.map(loader, num_parallel_calls=tf.data.AUTOTUNE)
        if shuffle:
            ds = ds.shuffle(buffer_size=len(labels), seed=random_state)
        return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    train_ds = make_ds(X_train, S_train, y_train, shuffle=True)
    val_ds   = make_ds(X_val,   S_val,   y_val,   shuffle=False)
    test_ds  = make_ds(X_test,  S_test,  y_test,  shuffle=False)

    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = load_and_split_multimodal_data(
    sensor_csv_path= "../dataset/Gas Sensors Measurements/Gas_Sensors_Measurements.csv",
    image_data_dir= "../dataset/Thermal Camera Images")

In [ ]:
class RandomSensorDropout(tf.keras.layers.Layer):
    """
    Layer that randomly zeros individual sensor channels with a given rate during training.
    Supports proper serialization.
    """
    def __init__(self, rate=0.3, **kwargs):
        super().__init__(**kwargs)
        self.rate = rate

    def call(self, inputs, training=False):
        if training and self.rate > 0.0:
            mask = tf.cast(tf.random.uniform(tf.shape(inputs)) > self.rate, inputs.dtype)
            return inputs * mask
        return inputs

    def get_config(self):
        config = super().get_config()
        config.update({"rate": self.rate})
        return config
    

# --------------- Monte Carlo Dropout Layers ---------------------    
class MCDropout(tf.keras.layers.Dropout):
    """
    Dropout that is active both at train *and* inference time,
    so we can sample N stochastic forward passes.
    """
    def call(self, inputs, training=None):
        # Force dropout even in inference
        return super().call(inputs, training=True)
        
class MCSpatialDropout2D(tf.keras.layers.SpatialDropout2D):
    """
    Dropout that is active both at train *and* inference time,
    so we can sample N stochastic forward passes.
    """
    def call(self, inputs, training=None):
        # Force dropout even in inference
        return super().call(inputs, training=True)



In [ ]:
def build_multimodal_model(
    img_shape=(120, 160, 3),
    input_dim=7,
    sensor_dropout_rate=0.3,
    layer_dropout=0.5,
    sensor_units=32,
    img_dense=64,
    fusion_dense=64,
    output_units=4,
    lr=1e-4
):
    """
    Intermediate fusion of sensor and image branches.
    Enhanced image branch with deeper layers and normalization.
    """
    # Sensor input
    s_in = tf.keras.Input(shape=(input_dim,), name="sensor_input")
    s = RandomSensorDropout(sensor_dropout_rate, name="sensor_dropout")(s_in)
    s = tf.keras.layers.Dense(sensor_units, activation="relu")(s)
    s = MCDropout(layer_dropout)(s)

    # Image input
    i_in = tf.keras.Input(shape=img_shape, name="image_input")
    x = tf.keras.layers.Conv2D(32, 3, padding="same", activation="relu" ,use_bias=False)(i_in)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(64, 3, padding="same", activation="relu", use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(128, 3, padding="same", activation="relu", use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = MCSpatialDropout2D(0.2)(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(img_dense, activation="relu")(x)
    x = MCDropout(layer_dropout)(x)

    # Fusion
    fused = tf.keras.layers.Concatenate()([s, x])
    y = tf.keras.layers.Dense(fusion_dense, activation="relu")(fused)
    y = MCDropout(layer_dropout)(y)
    out = tf.keras.layers.Dense(output_units, activation="softmax", name="output")(y)

    # Model compile
    model = tf.keras.Model([s_in, i_in], out, name="multimodal")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


In [ ]:
import matplotlib.pyplot as plt

def predict_mc(model, dataset, T=50):
    all_preds = []
    all_labels = []
    for (img_batch, sens_batch), lbl_batch in dataset:
        # run T stochastic passes
        preds_t = [
            model((img_batch, sens_batch), training=True).numpy()
            for _ in range(T)
        ]
        # stack & average: (T, batch, classes) → (batch, classes)
        mean_preds = np.stack(preds_t, axis=0).mean(axis=0)
        all_preds.append(mean_preds)
        all_labels.append(lbl_batch.numpy())

    all_preds  = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    y_pred = np.argmax(all_preds, axis=1)
    return np.mean(y_pred == all_labels)

dropout_rates = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5 , 0.6 , 0.7 , 0.8 , 0.9]
val_accuracies = []

for rate in dropout_rates:
    model = build_multimodal_model(input_dim=7, sensor_dropout_rate=rate)
    ckpt_path = f"best_sensor_dropout_{rate:.1f}.keras"
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        ckpt_path,
        monitor="val_accuracy",
        mode="max",
        save_best_only=True,
        verbose=0
    )
    early_stop_cb = EarlyStopping(
    monitor="val_accuracy",   # same metric as the checkpoint
    mode="max",
    patience=10,               # stop after 10 epochs with no improvement
    restore_best_weights=True # load best weights back into the model
    )
        
    model.fit(train_ds, validation_data=val_ds, epochs=100 , callbacks=[checkpoint_cb, early_stop_cb])
    best_model = tf.keras.models.load_model(
        ckpt_path,
        custom_objects={"RandomSensorDropout": RandomSensorDropout , "MCDropout": MCDropout , "MCSpatialDropout2D": MCSpatialDropout2D}
    )
    acc = predict_mc(best_model, test_ds, T=50)
    val_accuracies.append(acc)
    print(f"Dropout={rate:.1f} → MC Test Accuracy = {acc:.4f}")

# Plot dropout rate vs MC test accuracy
plt.figure(figsize=(8, 5))
plt.plot(dropout_rates, val_accuracies, marker='o')
plt.title("Sensor Dropout Rate vs. MC Test Accuracy for Multimodal")
plt.xlabel("Sensor Dropout Rate")
plt.ylabel("MC Test Accuracy")
plt.xticks(dropout_rates)
plt.grid(True)
plt.show()